In [16]:
import pandas as pd
import numpy as np
import pymongo

pd.set_option('display.max_columns', 500)

In [7]:
db_name = 'guide_data'
service_collection_name = 'service'
company_collection_name = 'company'
deals_collection_name = 'deal'

conn = pymongo.MongoClient("mongodb://{}:{}@{}:{}/{}?authSource=admin".format('mongoadmin', '12345', 'localhost', '27017', db_name))

In [8]:
service_collection = lambda c: c[db_name][service_collection_name]
company_collection = lambda c: c[db_name][company_collection_name]
deals_collection = lambda c: c[db_name][deals_collection_name]

In [46]:
companies = pd.DataFrame(company_collection(conn).find())
cat_cols = ['got_support_from', 'did_get_support', 'service', 'stage_of_development', 'msp_category', 'is_export', 'inno_cluster_member', 'skolcovo_member', 'is_inno_company', 'is_startup']
companies[cat_cols] = companies[cat_cols].astype('category')
companies.drop(columns=['_id'], inplace=True)
companies.replace(['', 'н.д.'], np.nan, inplace=True)

In [49]:
companies.shape

(7347, 22)

In [50]:
deals = pd.DataFrame(deals_collection(conn).find())
deals['round'] = deals['round'].astype('category')
deals.drop(columns=['_id'], inplace=True)
deals.replace(['', 'н.д.'], np.nan, inplace=True)

In [51]:
deals.shape

(940, 24)

In [64]:
services = pd.DataFrame(service_collection(conn).find())
services.drop(columns=['_id'], inplace=True)
services.replace(['', 'н.д.'], np.nan, inplace=True)

In [65]:
services.shape

(498, 25)

In [53]:
deals.head()

,deal_date,startup_name,startup_inn,gov_fund,gov_fund_sum_dol,gov_fund_sum_rub,private_fund,private_fund_sum_dol,private_fund_sum_rub,corp_fund,corp_fund_sum_dol,corp_fund_sum_rub,corp_investor,corp_investor_sum_dol,corp_investor_sum_rub,business_angel,business_angel_sum_dol,business_angel_sum_rub,accelerator,accelerator_sum_dol,accelerator_sum_rub,final_price_dol,final_price_rub,round
0,2017-01-01,leadza,7721379557,NaN,0.0,0.0,фрии,2.515517e+05,1.500000e+07,NaN,0.000000,0.0,NaN,0.0,0.0,NaN,0.000000,0.0,NaN,0.0,0.0,2.515517e+05,0.0,seed
1,2017-01-01,трансинфотех,7714858580,NaN,0.0,0.0,NaN,0.000000e+00,0.000000e+00,коммит кэпитал,670804.411881,40000000.0,NaN,0.0,0.0,NaN,0.000000,0.0,NaN,0.0,0.0,6.708044e+05,0.0,seed
2,2017-01-01,easy law,7727793108,NaN,0.0,0.0,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,0.0,NaN,0.0,0.0,александр румянцев,8385.055149,500000.0,NaN,0.0,0.0,8.385055e+03,0.0,pre-seed
3,2017-01-01,голод,NaN,NaN,0.0,0.0,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,0.0,NaN,0.0,0.0,"михаил соколов, бизнес-ангелы",167701.102970,10000000.0,NaN,0.0,0.0,1.677011e+05,0.0,seed
4,2017-01-01,mokka,7724856208,NaN,0.0,0.0,"baring vostok capital partners, vostok emergin...",2.000000e+07,1.192598e+09,NaN,0.000000,0.0,NaN,0.0,0.0,NaN,0.000000,0.0,NaN,0.0,0.0,2.000000e+07,0.0,в


In [58]:
present_inns = set(deals['startup_inn']) & set(companies['inn'])

In [59]:
deals[deals['startup_inn'].apply(lambda x: x in present_inns)][['gov_fund', 'private_fund', 'corp_fund', 'corp_investor', 'business_angel', 'accelerator']].count()

gov_fund           50
private_fund      154
corp_fund          32
corp_investor      60
business_angel    108
accelerator        65
dtype: int64

In [68]:
services.head()

,inn,name,startup_stage,market,services,technologies,type_of_ownership,investment_round,investition_from_dol,investition_to_dol,tech_focus,fund_total_rub,fund_total_dol,num_of_investments,num_of_exits,geography,study_format,num_of_people_in_company_from,num_of_people_in_company_to,num_of_participants,okved_main,okved_secondary,corp_stage,business_model,monetary_support
0,7703638456,название фонда - da vinci capital technology f...,"[ранний рост, расширение]","[cybersecurity, transport & logistics, busines...",[инвестиции],"[ar/vr, big data, блокчейн, интернет вещей, ис...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7728461088,NaN,"[ранний рост, расширение, посевная]",[cleantech],[инвестиции],"[3d моделирование, big data, интернет вещей, н...",частный,"[seed, раунд а, раунд в]",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,название фонда - addventure v\nобъем инвестици...,"[ранний рост, расширение]","[hrtech, consumer goods & services, foodtech, ...",[инвестиции],"[3d моделирование, ar/vr, big data, аддитивные...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,10000000.0,[],0.0,0.0,45.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,"[ранний рост, посевная]","[business software, advertising & marketing, e...","[инвестиции, консультационная поддержка по раз...","[ar/vr, искусственный интеллект и машинное обу...",корпоративный,"[seed, раунд а]",200000.0,2000000.0,[],0.0,0.0,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,название фонда - altair capital fund iii\nобъе...,"[ранний рост, расширение, посевная]","[edtech, insurancetech, business software, e-c...","[инвестиции, консультационная поддержка по раз...","[блокчейн, искусственный интеллект и машинное ...",частный,"[seed, раунд а, раунд с+]",100000.0,10000000.0,[],35000.0,500.0,271.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
